In [101]:
import nibabel as nib
import numpy as np
from sklearn.linear_model import Lasso,LassoCV,LinearRegression,Ridge
from sklearn.decomposition import TruncatedSVD, KernelPCA, PCA
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from scipy.ndimage.interpolation import zoom
from math import sqrt
from sklearn.ensemble import AdaBoostRegressor,BaggingRegressor, RandomForestRegressor
from mpl_toolkits.mplot3d import Axes3D
from math import sqrt,atan2,pi,ceil,acos,sin,cos
from sklearn.metrics import hamming_loss

from orelmisc import n_max,n_test_max,testpre,trainpre,saveCSV,shape_3d_org
from preprocess import flatten,flatten_each_sample,loadData,flatten_each_sample,precompute_and_save

In [102]:
prep_ = lambda i: cap(i)
def prep(a):
  return map(prep_single,a)

prep_single = lambda pi:map(prep_,pi)

cap = lambda p: 0 if min(1,max(0,p)) < 0.5 else 1
def pred_to_rgb(p):
  p = cap(p)
  v = p
  return (1-v,v,0.2)

y = np.zeros((n_max,3))
# gender: 1 = female, 0 = male
# age: 1 = young, 0 = old
# health: 1 = healthy, 0 = sick
y_gender = [0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
y_age = [1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
y_health = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
y[:,0] = np.array(y_gender)
y[:,1] = np.array(y_age)
y[:,2] = np.array(y_health)

result = None
xa = []
x = None
x_t = None
TO_RADIANS = pi/180

In [103]:
space = 70 #35 # 35,50,70,100,150
bins = np.linspace(1,1700,space)
hSize = space-1
r_division = 6
theta_division = 4
phi_division = 6
ranges = [1,r_division,theta_division,phi_division]
nHists = r_division*theta_division*phi_division

# IDEA:
# * take a look at the empirical distribution (Anteile) of the predicted classes.
#   All the real-valued predictions can be plotted in a 3D plot :)
#   If there are too few (many) predictions in a class (by comparision to Anteile in training set),
#   then move the closest other samples to our class (furthest of current samples to another class).

name = "output/"+str(space)+"_split_validated"
fname = "many_hists"+str(nHists)+"_space" + str(space) + ("_divs_%s_%s_%s.npy" % (r_division,theta_division,phi_division))
fnameSpherical = "spherical_every2.npy"
fnameMeanHist = "mean_hist" + str(space) + ".npy"
fnameMeanHists = "mean_hists" + str(space) + ".npy"

xSpherical = np.load(fnameSpherical)
sbins = bins[:-1]+1800/space/2
ds = nHists*hSize # sum,sum2,sum3,avg,var,mode, numper of percentiles and histograms
iHist = 0
idxHist = lambda i: iHist+i*hSize
# radius form 0 to 80
# theta from 0 to 180
# phi from 0 to 360
rMax = 80
tMin = 30
tMax = 180-tMin

print "Number of Histograms:",nHists
print "Size of Histograms:",hSize
print "Dimensions: ca.",nHists*hSize

Number of Histograms: 144
Size of Histograms: 69
Dimensions: ca. 9936


In [104]:
def myloop(a):
    return (a,a*a,a*a*a)

def spherical2cart(r,theta,phi):
  return (r*sin(theta)*cos(phi),r*sin(theta)*sin(phi),r*cos(theta))

myloopU = np.frompyfunc(myloop,1,3,dtype=np.float64)
fromto = lambda di,sph: zip(np.round(np.linspace(0,sph.shape[di],ranges[di]+1)),np.round(np.linspace(0,sph.shape[di],ranges[di]+1))[1:])

def loadAndPreprocess():
    global d_org,x,x_t,xa,x_org,x_t_org,xa_org,xSpherical
    
    xa = np.zeros((n_max+n_test_max,ds))
    #xSpherical = np.zeros((n_max+n_test_max,rMax/2+1,(tMax-tMin)/2+1,360/2+1))
    
    i = 0
    #print "===== Calculate spherical coordiantes ===="
    
    while i < n_max+n_test_max:
      if i % 6 == 0:
        print "  i = %s ... %.1f%%" % (i,float(i)/(n_max+n_test_max)*100)
      
      n_i,pre,t_str = (n_max,"set_train/","train") if i < n_max else (n_test_max,"set_test/","test")
      filename = "%s%s_%s.nii" % (pre,t_str,i%n_max+1)
      Xtotal,Ytotal,Ztotal = (176,208,176)
      data = nib.load(filename).get_data().reshape((Xtotal,Ytotal,Ztotal))
      
      """Calculate zoomed spherical representation, needs 3 seconds
      for r in np.linspace(0,rMax,rMax/2+1):
        for theta in np.linspace(tMin,tMax,(tMax-tMin)/2+1):
          for phi in np.linspace(0,360,360/2+1):
            x,y,z = spherical2cart(r,theta*TO_RADIANS,phi*TO_RADIANS)
            x = x + Xtotal/2
            y = y + Ytotal/2
            z = z + Ztotal/2
            if 0 <= x < Xtotal and 0 <= y < Ytotal and 0 <= z < Ztotal:
              xSpherical[i,r/2,(theta-tMin)/2,phi/2] = data[int(x),int(y),int(z)]"""

      # calculate histograms: 8*4*8 = 256 histograms
      hCount = 0
      for l,u in fromto(1,xSpherical):
        l0,u0=(int(l),int(u))
        for l,u in fromto(2,xSpherical):
          l1,u1=(int(l),int(u))
          for l,u in fromto(3,xSpherical):
            l2,u2=(int(l),int(u))
            cut = xSpherical[i,l0:u0,l1:u1,l2:u2]
            h=np.histogram(cut.ravel(),bins=bins)[0]
            xa[i,(iHist+(hCount*hSize)):(iHist+(hCount+1)*hSize)] = h
            hCount = hCount + 1
      i = i+1
    
    np.save(fname,xa)
    print "======= Saved data matrix xa into %s =========" % fname
    
    #np.save(fnameSpherical,xSpherical)
    #print "======= Saved spherical coordinates into %s =========" % fnameSpherical

if False:
  loadAndPreprocess()
else:
  xa = np.load(fname)
  x,x_t = (xa[0:n_max,:],xa[n_max:,:])

In [105]:
if False:
  i = 413
  for theta in np.linspace(0,xSpherical[i].shape[1]-1,9):
    print theta
    plt.imshow(xSpherical[i,:,theta,:], cmap="gray", origin="lower")
    plt.show()

In [106]:
from sklearn.model_selection import train_test_split

# xa. all data
# x. public training data
# y. public training targets
# x_t public to be predicted data

# xtr. training data
# ytr. training targets
# xts. validation data
# yts. validation targets

b = [0,1]
types = [ [a1,a2,a3] for a1 in b for a2 in b for a3 in b]
Ntypes = len(types)

def get_by(t,ys):
    idxs = range(0,len(ys))
    return filter(lambda i:list(ys[i,:])==list(t),idxs)

#indices for splitting
typeIdxs = [get_by(t,y) for t in types]

typeIdxsTR = []
typeIdxsTS = []

for i in range(0,Ntypes):
    atr,ats,_,_ = train_test_split(typeIdxs[i],np.zeros(len(typeIdxs[i])),test_size=0.3,random_state=1)
    typeIdxsTR.append(atr)
    typeIdxsTS.append(ats)
    
typeIdxs = np.array(typeIdxs)
typeIdxsTR = np.array(typeIdxsTR)
typeIdxsTS = np.array(typeIdxsTS)

# gender: 1 = female, 0 = male
# age: 1 = young, 0 = old
# health: 1 = healthy, 0 = sick

def show_diff_for(name,es1,es2,total):
  es = map(lambda x,y:x-y,es1,es2)
  abses = map(abs,es)
  print name,"contains",sum(abses),"samples of total",total,": %0.2f%%" % (100*float(sum(abses))/total)
  for i in range(0,8):
      tp = ((100*float(es[i])/total),types[i],es[i])
      print "  %0.2f%%: %s with samples: #%s" % tp
      
def show_diff(name,es1,es2):
  es1 = map(len,es1)
  es2 = map(len,es2)
  show_diff_for(name,es1,es2,total=sum(es1))
        
def show_for(name,es):
  es1 = map(lambda x:len(x),es)
  show_diff_for(name,es1,[0]*8,sum(es1))

show_for("public Training",typeIdxs)
show_for("our Training",typeIdxsTR)
show_for("our Test",typeIdxsTS)

xtr = np.vstack([x[idxs]for idxs in typeIdxsTR])
xts = np.vstack([x[idxs]for idxs in typeIdxsTS])
ytr = np.vstack([y[idxs]for idxs in typeIdxsTR])
yts = np.vstack([y[idxs]for idxs in typeIdxsTS])

print "Splitted data into test and validation data"

public Training contains 278 samples of total 278 : 100.00%
  10.07%: [0, 0, 0] with samples: #28
  6.47%: [0, 0, 1] with samples: #18
  0.00%: [0, 1, 0] with samples: #0
  22.30%: [0, 1, 1] with samples: #62
  14.03%: [1, 0, 0] with samples: #39
  17.27%: [1, 0, 1] with samples: #48
  0.00%: [1, 1, 0] with samples: #0
  29.86%: [1, 1, 1] with samples: #83
our Training contains 192 samples of total 192 : 100.00%
  9.90%: [0, 0, 0] with samples: #19
  6.25%: [0, 0, 1] with samples: #12
  0.00%: [0, 1, 0] with samples: #0
  22.40%: [0, 1, 1] with samples: #43
  14.06%: [1, 0, 0] with samples: #27
  17.19%: [1, 0, 1] with samples: #33
  0.00%: [1, 1, 0] with samples: #0
  30.21%: [1, 1, 1] with samples: #58
our Test contains 86 samples of total 86 : 100.00%
  10.47%: [0, 0, 0] with samples: #9
  6.98%: [0, 0, 1] with samples: #6
  0.00%: [0, 1, 0] with samples: #0
  22.09%: [0, 1, 1] with samples: #19
  13.95%: [1, 0, 0] with samples: #12
  17.44%: [1, 0, 1] with samples: #15
  0.00%: [1,

In [129]:
def applySingleClassification(classf,label):
  prefix_str,model = classf
  xtr1 = xtr[:,:]
  xts1 = xts[:,:]
  x_t1 = x_t[:,:]
  
  yts_pred = model.fit(xtr1,ytr[:,label]).predict(xts1)
  y_t_pred = model.predict(x_t1)
  
  y_t_pp = prep_single(y_t_pred)
  yts_pp = prep_single(yts_pred)
  ytr_pp = prep_single(model.predict(xtr1))
  
  trCorrect = len(filter(lambda x:x,map(lambda x,y:x==y,ytr[:,label],ytr_pp)))
  tsCorrect = len(filter(lambda x:x,map(lambda x,y:x==y,yts[:,label],yts_pp)))
  ltr = 100*hamming_loss(ytr[:,label],ytr_pp)
  lts = 100*hamming_loss(yts[:,label],yts_pp)
  
  print "|   %3d    /    %3d    |   %3d   /     %3d    |    %3.1f%%     |     %3.1f%%     |" % (trCorrect,len(xtr)-trCorrect,tsCorrect,len(xts)-tsCorrect,ltr,lts)
 #print "|  correct / incorrect | correct / incorrect | Hamming Loss | lts =        |"
  return model,y_t_pp,yts_pp,ltr,lts

def applyClassificationSeparated(classf):
  prefix = classf[0]
  # TODO: copy output to file
  print "### Prediction with ###\n","  ",prefix,"\n"
  
  print "| Predictions training | Predictions test     | Hamming Loss | Hamming Loss |"
  print "|  correct / incorrect | correct / incorrect  |   Training   |    Test      |"
  
  model1,y_gender_pred,ytspp1,ltr1,lts1 = applySingleClassification(classf,0)
  model2,y_age_pred   ,ytspp2,ltr2,lts2 = applySingleClassification(classf,1)
  model3,y_health_pred,ytspp3,ltr3,lts3 = applySingleClassification(classf,2)
  
  y_pred_all = np.array([y_gender_pred,y_age_pred,y_health_pred]).transpose()
  y_pred_ts = np.array([ytspp1,ytspp2,ytspp3]).transpose()
  ltr = np.average([ltr1,ltr2,ltr3])
  lts = np.average([lts1,lts2,lts3])
  
  print "|                     Total                   |    %3.1f%%     |     %3.1f%%     |" % (ltr,lts)
  
  return (y_pred_all,y_pred_ts,ltr,lts)

In [130]:
def doStuff(al=15,comps=n_max-1,n_est=120,k1="rbf",
            max_depth=7,msplit=10,feats=0.3,samps=0.3,rs=0,
            kernel="rbf",coef0=1,p=False,deg=3,gamma=1.0/600,C=80,n_comps=15,f=0.5,msp=15,hmin=0,hmax=50):
  #prefix,model = makeLassoPrediction(al=al)
  #prefix,model = makeAdaDefaultBaseEstimatorPrediction(n_est=n_est,max_depth=max_depth,msplit=msplit)
  #prefix,model = makeAdaLassoPrediction(al=al,n_est=n_est)
  #prefix,model = kernelSVM(kernel=kernel,coef0=coef0,probability=p,degree=deg,gamma=gamma,C=C)
  #prefix,model = kernelSVMwithDimReduction(kernel=kernel,k1=k1,coef0=coef0,probability=p,degree=deg,gamma=gamma,C=C,n_comps=n_comps)
  #prefix,model = kSVM_Ada(n_est=n_est,kernel=kernel,k1=k1,coef0=coef0,probability=p,degree=deg,gamma=gamma,C=C,n_comps=n_comps)
  #prefix,model = makeBaggingBoostLassoPrediction(al=al,n_est=n_est,feats=feats,samps=samps)
  #prefix,model = makeSVDLassoPrediction(al=al,comps=comps)
  #prefix,model = makeLassoCVPrediction(cv=3)
  #prefix,model = makeLinearRegression()
  #prefix,model = makeRidgePrediction(al=al)
  #prefix,model = makeGaussianProcess()
  # Bagging Lasso works poorly when validated and on the public score. forget this.
  #prefix,model = makeBaggingBoostLassoPrediction(al=al,n_est=n_est,feats=feats,samps=samps)
  
  prefix = "%s_RandomForest_histMin%s_histMax%s_n%s_feats%s_msp%s_max_depth%s"%(name,hmin,hmax,n_est,f,msp,max_depth)
  model = RandomForestRegressor(n_est,max_features=f,min_samples_split=msp,max_depth=max_depth,random_state=rs)
  
  y_pred_all,y_pred_ts,ltr,lts = applyClassificationSeparated((prefix,model))
  
  eachX_T = np.array([ get_by(t,y_pred_ts) for t in types])
  #show_for("our Test",typeIdxsTS)
  #show_for("predicted Test",eachX_T)
  show_diff("Error",typeIdxsTS,eachX_T)
  
  visualize("Gender",y_pred_all[:,0])
  visualize("Age   ",y_pred_all[:,1])
  visualize("Health",y_pred_all[:,2])
  
  savePrediction(y_pred_all,prefix)
  
  result = (x,y,x_t,y_pred_all,y_pred_all,y_pred_ts)

def visualize(labelName,y_pred):
  total = int(len(y_pred)*0.7)
  zeros = int(len(filter(lambda x:x == 0,y_pred))*0.7)
  print labelName," ","0"*zeros,"#","1"*(total-zeros),"***"

def savePrediction(ys,prefix):
    savedFilename = saveCSV(ys,prefix)
    print("Saved predictions into %s" % savedFilename)

In [136]:
doStuff(
  #kernel="poly",gamma=1.0/2,coef0=1,deg=2,al=9,k1="poly",n_comps=15,C=1,n_est=70,
  n_est=250,f=0.07,msp=10,max_depth=3,hmin=0,hmax=nHists,rs=2,
  msplit=70)

### Prediction with ###
   output/70_split_validated_RandomForest_histMin0_histMax144_n250_feats0.07_msp10_max_depth3 

| Predictions training | Predictions test     | Hamming Loss | Hamming Loss |
|  correct / incorrect | correct / incorrect  |   Training   |    Test      |
|   186    /      6    |    61   /      25    |    3.1%     |     29.1%     |
|   192    /      0    |    78   /       8    |    0.0%     |     9.3%     |
|   191    /      1    |    73   /      13    |    0.5%     |     15.1%     |
|                     Total                   |    1.2%     |     17.8%     |
Error contains 42 samples of total 86 : 48.84%
  5.81%: [0, 0, 0] with samples: #5
  6.98%: [0, 0, 1] with samples: #6
  0.00%: [0, 1, 0] with samples: #0
  11.63%: [0, 1, 1] with samples: #10
  -11.63%: [1, 0, 0] with samples: #-10
  -5.81%: [1, 0, 1] with samples: #-5
  0.00%: [1, 1, 0] with samples: #0
  -6.98%: [1, 1, 1] with samples: #-6
Gender   0000000000 # 1111111111111111111111111111111111111111111111